<a href="https://colab.research.google.com/github/nikatnguyen/Project4/blob/main/anika_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [2]:
!pip install keras-tuner

In [3]:
#Import dataset
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/data class notebooks/clean_df.csv')
df.head()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Insufficient_Weight,Normal_Weight,...,yes.3,Always.1,Frequently.1,Sometimes.1,no.5,Automobile,Bike,Motorbike,Public_Transportation,Walking
0,21.0,1.62,64.0,2.0,3.0,2.0,0.0,1.0,0,1,...,0,0,0,0,1,0,0,0,1,0
1,21.0,1.52,56.0,3.0,3.0,3.0,3.0,0.0,0,1,...,1,0,0,1,0,0,0,0,1,0
2,23.0,1.80,77.0,2.0,3.0,2.0,2.0,1.0,0,1,...,0,0,1,0,0,0,0,0,1,0
3,27.0,1.80,87.0,3.0,3.0,2.0,2.0,0.0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,22.0,1.78,89.8,2.0,1.0,2.0,0.0,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [5]:
#Split data set into features and targets
y = df[['Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III']].values
X = df.drop(columns=['Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III'])

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

ATTEMPT 1

In [7]:
#Defining model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="sigmoid", input_dim=35))
nn_model.add(tf.keras.layers.Dense(units=16, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=3, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
50/50 [==============================] - 2s 2ms/step - loss: 0.6165 - accuracy: 0.6968
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 0.5746 - accuracy: 0.6968
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 0.5456 - accuracy: 0.6968
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 0.5243 - accuracy: 0.6968
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.5078 - accuracy: 0.6968
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.4942 - accuracy: 0.6968
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 0.4830 - accuracy: 0.6968
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 0.4733 - accuracy: 0.6968
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 0.4652 - accuracy: 0.6968
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 0.4581 - accuracy: 0.6968
Epoch 11/50
50/50 [

ATTEMPT 2

In [8]:
#Defining model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=35))
nn_model.add(tf.keras.layers.Dense(units=16, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=3, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
50/50 [==============================] - 1s 2ms/step - loss: 0.6587 - accuracy: 0.6955
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 0.4941 - accuracy: 0.7265
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 0.4094 - accuracy: 0.7871
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3624 - accuracy: 0.7770
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3285 - accuracy: 0.7738
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2984 - accuracy: 0.7776
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2704 - accuracy: 0.8143
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2449 - accuracy: 0.8433
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2229 - accuracy: 0.8604
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2042 - accuracy: 0.8781
Epoch 11/50
50/50 [

ATTEMPT 3

In [9]:
#Defining model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=10, activation="relu", input_dim=35))
nn_model.add(tf.keras.layers.Dense(units=16, activation="sigmoid"))
nn_model.add(tf.keras.layers.Dense(units=3, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
50/50 [==============================] - 1s 2ms/step - loss: 0.5898 - accuracy: 0.3032
Epoch 2/50
50/50 [==============================] - 0s 2ms/step - loss: 0.4778 - accuracy: 0.4043
Epoch 3/50
50/50 [==============================] - 0s 2ms/step - loss: 0.4117 - accuracy: 0.4801
Epoch 4/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3660 - accuracy: 0.5445
Epoch 5/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3314 - accuracy: 0.5793
Epoch 6/50
50/50 [==============================] - 0s 2ms/step - loss: 0.3035 - accuracy: 0.6267
Epoch 7/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2800 - accuracy: 0.6380
Epoch 8/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2592 - accuracy: 0.6728
Epoch 9/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2400 - accuracy: 0.7088
Epoch 10/50
50/50 [==============================] - 0s 2ms/step - loss: 0.2224 - accuracy: 0.7391
Epoch 11/50
50/50 [